# Financial Planning with APIs and Simulations

In this Challenge, you’ll create two financial analysis tools by using a single Jupyter notebook:

Part 1: A financial planner for emergencies. The members will be able to use this tool to visualize their current savings. The members can then determine if they have enough reserves for an emergency fund.

Part 2: A financial planner for retirement. This tool will forecast the performance of their retirement portfolio in 30 years. To do this, the tool will make an Alpaca API call via the Alpaca SDK to get historical price data for use in Monte Carlo simulations.

You’ll use the information from the Monte Carlo simulation to answer questions about the portfolio in your Jupyter notebook.



In [32]:
# Import the required libraries and dependencies
import os
import requests
import json
# import time

import pandas as pd
import alpaca_trade_api as tradeapi

from datetime import datetime
from dateutil import tz
from dotenv   import load_dotenv
from monte_carlo_sim.MCForecastTools import MCSimulation
from monte_carlo_sim.alpaca.get_alpaca_daterange import dateranges

%matplotlib inline


In [33]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()


True

## Part 1: Create a Financial Planner for Emergencies

### Evaluate the Cryptocurrency Wallet by Using the Requests Library

In this section, you’ll determine the current value of a member’s cryptocurrency wallet. You’ll collect the current prices for the Bitcoin and Ethereum cryptocurrencies by using the Python Requests library. For the prototype, you’ll assume that the member holds the 1.2 Bitcoins (BTC) and 5.3 Ethereum coins (ETH). To do all this, complete the following steps:

1. Create a variable named `monthly_income`, and set its value to `12000`.

2. Use the Requests library to get the current price (in US dollars) of Bitcoin (BTC) and Ethereum (ETH) by using the API endpoints that the starter code supplies.

3. Navigate the JSON response object to access the current price of each coin, and store each in a variable.

    > **Hint** Note the specific identifier for each cryptocurrency in the API JSON response. The Bitcoin identifier is `1`, and the Ethereum identifier is `1027`.

4. Calculate the value, in US dollars, of the current amount of each cryptocurrency and of the entire cryptocurrency wallet.



In [34]:
def get_et_datetime(client_datetime, strformat):
    # Find the set timezone and convert to "America/New_York" (ET).
    to_zone     = tz.gettz('America/New_York')
    et_datetime = client_datetime.astimezone(to_zone)
    
    return et_datetime.strftime(strformat)


def combine_shares_title_str(shares):    
    share_strs = [" ".join([f"{s:.1f}", t]) for t, s in shares.items()]
    share_strs = " and ".join(share_strs)
    
    return share_strs


def combine_coins_title_str(coins):    
    coin_strs = [" ".join([f"{s:.2f}", t]) for t, s in coins.items()]
    coin_strs = " and ".join(coin_strs)
    
    return coin_strs


def closing_price_print(day, shares, title, value, portfolio):
    print(f"""
As of {day}, 
the closing price of the {portfolio}'s
{shares:.1f} {title} is {value:,.2f} USD.
    """)
    
    
def closing_value_print(day, shares, title, value, portfolio):
    print(f"""
As of {day}, 
the total value of the {portfolio}'s
{shares:.1f} {title} is {value:,.2f} USD.
    """)
    
    
def closing_portfolio_balance_print(day, value, portfolio, coins=None, shares=None):
    share_strs = ""
    coin_strs  = ""
    if shares:
        share_strs += combine_shares_title_str(shares) + " shares"
        
    if coins:
        coin_strs  += combine_shares_title_str(coins)  + " coins"
        
    share_strs = ", \nand ".join(filter(None, [share_strs, coin_strs]))
    
    print(f"""
As of {day},
the total closing value of the {portfolio} containing 
{share_strs} 
is {value:,.2f} USD.
    """)


In [35]:
class Investment():
    def __init__(self, ticker: str, name: str, count: float,
                 currency: str, portfolio: str, time_format: str):
        self.ticker          = ticker
        self.name            = name
        self.__count         = count
        self.currency        = currency
        self.portfolio       = portfolio
        self.time_format     = time_format
        self.__set_title()
        self.__set_id("")
        self.__set_collection_time(0)
        self.__set_price(0)
        self.__set_value(0)

    def __set_title(self):
        self.title = f"{self.name} ({self.ticker})"
    
    def __set_id(self, id: str):
        if id:
            self.id = id
        else:
            self.id = None
        
    def __set_collection_time(self, timestamp: datetime):
        if timestamp:
            self.collection_time = get_et_datetime(timestamp, self.time_format)
        else:
            self.collection_time = None
        
    def __set_price(self, price: float):
        if price:
            self.price = price
        else:
            self.price = None
            
    def __set_value(self, value: float):
        if value:
            self.value = value
        else:
            self.value = None
        
    def closing_price_print(self):
        closing_price_print(self.collection_time, 
                            self.__count, 
                            self.title, 
                            self.price, 
                            self.portfolio)
        
    def closing_value_print(self):
        closing_value_print(self.collection_time, 
                            self.__count, 
                            self.title, 
                            self.value, 
                            self.portfolio)
        
    @staticmethod
    def closing_portfolio_balance_print(collection_time,
                                        value,
                                        portfolio,
                                        coins=None,
                                        shares=None):        
        
        if coins:
            if not isinstance(coins, list): coins = [coins,]
            coins  = {c.title: c.coins  for c in coins}
            
        if shares:
            if not isinstance(shares, list): shares = [shares,]
            shares = {s.title: s.shares for s in shares}
           
        closing_portfolio_balance_print(collection_time,
                                        value,
                                        portfolio,
                                        coins=coins,
                                        shares=shares)


In [36]:
class FreeCryptoResponse(Investment):
    def __init__(self, ticker: str, name: str, coins: float, currency: str):
        super().__init__(ticker,
                         name,
                         coins,
                         currency,
                         "cryptocurrency wallet",
                         "%a, %Y-%b-%d %H:%M:%S (%Z)")
        
        self.coins = coins
        self.set_response({})
        
    def set_response(self, response: dict):
        if response:
            self.response = response
            
            id        = list(response["data"].keys())[0]
            timestamp = datetime.fromtimestamp(response["metadata"]["timestamp"])
            price     = float(response["data"][id]["quotes"]["USD"]["price"])
            value     = self.coins * price
                
            self._Investment__set_id(id)            
            self._Investment__set_collection_time(timestamp)
            self._Investment__set_price(price)
            self._Investment__set_value(value)
        else:
            self.response = response


In [37]:
class AlpacaBarset(Investment):
    def __init__(self, ticker: str, name: str, shares: float, currency: str):
        super().__init__(ticker,
                         name,
                         shares,
                         currency,
                         "stock and bond portion of the portfolio",
                         "%a, %Y-%b-%d (%Z)")
        
        self.shares = shares
        self.set_response(pd.DataFrame())
        
    def set_response(self, response: pd.DataFrame):
        if not response.empty:
            self.response = response.loc[:, self.ticker]
            
            id        = ""
            timestamp = response.iloc[-1].name
            price     = response.loc[timestamp, (self.ticker, "close")]
            value     = self.shares * price
                
            self._Investment__set_id(id)
            self._Investment__set_collection_time(timestamp)
            self._Investment__set_price(price)
            self._Investment__set_value(value)
        else:
            self.response = response


In [38]:
# The current number of coins for each cryptocurrency asset held in the portfolio.
btc_coins = 1.2
eth_coins = 5.3
        

#### Step 1: Create a variable named `monthly_income`, and set its value to `12000`.

In [39]:
# The monthly amount for the member's household income
monthly_income = 12000

btc = FreeCryptoResponse("BTC", "Bitcoin",  btc_coins, "USD")
eth = FreeCryptoResponse("ETH", "Ethereum", eth_coins, "USD")

btc.__dict__


{'ticker': 'BTC',
 'name': 'Bitcoin',
 '_Investment__count': 1.2,
 'currency': 'USD',
 'portfolio': 'cryptocurrency wallet',
 'time_format': '%a, %Y-%b-%d %H:%M:%S (%Z)',
 'title': 'Bitcoin (BTC)',
 'id': None,
 'collection_time': None,
 'price': None,
 'value': None,
 'coins': 1.2,
 'response': {}}

#### Review the endpoint URLs for the API calls to Free Crypto API in order to get the current pricing information for both BTC and ETH.

In [40]:
# The Free Crypto API Call endpoint URLs for the held cryptocurrency assets
btc_url = f"https://api.alternative.me/v2/ticker/{btc.name}/?convert={btc.currency}"
eth_url = f"https://api.alternative.me/v2/ticker/{eth.name}/?convert={eth.currency}"


#### Step 2. Use the Requests library to get the current price (in US dollars) of Bitcoin (BTC) and Ethereum (ETH) by using the API endpoints that the starter code supplied.

In [41]:
# Using the Python requests library, make an API call to access the current price of BTC
btc_response = requests.get(btc_url).json()
btc.set_response(btc_response)

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(btc_response, indent=4, sort_keys=True))


{
    "data": {
        "1": {
            "circulating_supply": 18764012,
            "id": 1,
            "last_updated": 1627010963,
            "max_supply": 21000000,
            "name": "Bitcoin",
            "quotes": {
                "USD": {
                    "market_cap": 612527898100,
                    "percent_change_1h": -0.0473490156374078,
                    "percent_change_24h": 1.80833843186153,
                    "percent_change_7d": 2.81720120976097,
                    "percentage_change_1h": -0.0473490156374078,
                    "percentage_change_24h": 1.80833843186153,
                    "percentage_change_7d": 2.81720120976097,
                    "price": 32620.0,
                    "volume_24h": 20228880402
                }
            },
            "rank": 1,
            "symbol": "BTC",
            "total_supply": 18764012,
            "website_slug": "bitcoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrenc

In [42]:
# Using the Python requests library, make an API call to access the current price ETH
eth_response = requests.get(eth_url).json()
eth.set_response(eth_response)

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(eth_response, indent=4, sort_keys=True))


{
    "data": {
        "1027": {
            "circulating_supply": 116803686,
            "id": 1027,
            "last_updated": 1627010960,
            "max_supply": 0,
            "name": "Ethereum",
            "quotes": {
                "USD": {
                    "market_cap": 241662762652,
                    "percent_change_1h": -0.214610913442622,
                    "percent_change_24h": 4.1936400217531,
                    "percent_change_7d": 7.86436845064656,
                    "percentage_change_1h": -0.214610913442622,
                    "percentage_change_24h": 4.1936400217531,
                    "percentage_change_7d": 7.86436845064656,
                    "price": 2060.99,
                    "volume_24h": 21958766349
                }
            },
            "rank": 2,
            "symbol": "ETH",
            "total_supply": 116803686,
            "website_slug": "ethereum"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrenci

#### Step 3: Navigate the JSON response object to access the current price of each coin, and store each in a variable.

In [43]:
# Navigate the BTC response object to access the current price of BTC
btc_price = float(btc.response["data"]["1"]["quotes"]["USD"]["price"])

# Print the current price of BTC
btc.closing_price_print()



As of Thu, 2021-Jul-22 23:29:23 (EDT), 
the closing price of the cryptocurrency wallet's
1.2 Bitcoin (BTC) is 32,620.00 USD.
    


In [44]:
# Navigate the BTC response object to access the current price of ETH
eth_price = float(eth_response["data"]["1027"]["quotes"]["USD"]["price"])

# Print the current price of ETH
eth.closing_price_print()



As of Thu, 2021-Jul-22 23:29:20 (EDT), 
the closing price of the cryptocurrency wallet's
5.3 Ethereum (ETH) is 2,060.99 USD.
    


### Step 4: Calculate the value, in US dollars, of the current amount of each cryptocurrency and of the entire cryptocurrency wallet.

In [45]:
# Compute the current value of the BTC holding 
btc_value = btc_coins * btc_price

# Print current value of your holding in BTC
btc.closing_value_print()



As of Thu, 2021-Jul-22 23:29:23 (EDT), 
the total value of the cryptocurrency wallet's
1.2 Bitcoin (BTC) is 39,144.00 USD.
    


In [46]:
# Compute the current value of the ETH holding 
eth_value = btc_coins * btc_price

# Print current value of your holding in ETH
eth.closing_value_print()



As of Thu, 2021-Jul-22 23:29:20 (EDT), 
the total value of the cryptocurrency wallet's
5.3 Ethereum (ETH) is 10,923.25 USD.
    


In [47]:
# Compute the total value of the cryptocurrency wallet
# Add the value of the BTC holding to the value of the ETH holding
total_crypto_wallet = btc_value + eth_value

# Print current cryptocurrency wallet balance
btc.closing_portfolio_balance_print(btc.collection_time,
                                    total_crypto_wallet,
                                    btc.portfolio,
                                    coins=[btc, eth])



As of Thu, 2021-Jul-22 23:29:23 (EDT),
the total closing value of the cryptocurrency wallet containing 
1.2 Bitcoin (BTC) and 5.3 Ethereum (ETH) coins 
is 78,288.00 USD.
    


### Evaluate the Stock and Bond Holdings by Using the Alpaca SDK

In this section, you’ll determine the current value of a member’s stock and bond holdings. You’ll make an API call to Alpaca via the Alpaca SDK to get the current closing prices of the SPDR S&P 500 ETF Trust (ticker: SPY) and of the iShares Core US Aggregate Bond ETF (ticker: AGG). For the prototype, assume that the member holds 110 shares of SPY, which represents the stock portion of their portfolio, and 200 shares of AGG, which represents the bond portion. To do all this, complete the following steps:

1. In the `Starter_Code` folder, create an environment file (`.env`) to store the values of your Alpaca API key and Alpaca secret key.

2. Set the variables for the Alpaca API and secret keys. Using the Alpaca SDK, create the Alpaca `tradeapi.REST` object. In this object, include the parameters for the Alpaca API key, the secret key, and the version number.

3. Set the following parameters for the Alpaca API call:

    - `tickers`: Use the tickers for the member’s stock and bond holdings.

    - `timeframe`: Use a time frame of one day.

    - `start_date` and `end_date`: Use the same date for these parameters, and format them with the date of the previous weekday (or `2020-08-07`). This is because you want the one closing price for the most-recent trading day.

4. Get the current closing prices for `SPY` and `AGG` by using the Alpaca `get_barset` function. Format the response as a Pandas DataFrame by including the `df` property at the end of the `get_barset` function.

5. Navigating the Alpaca response DataFrame, select the `SPY` and `AGG` closing prices, and store them as variables.

6. Calculate the value, in US dollars, of the current amount of shares in each of the stock and bond portions of the portfolio, and print the results.


#### Review the total number of shares held in both (SPY) and (AGG).

In [48]:
# Current amount of shares held in both the stock (SPY) and bond (AGG) portion of the portfolio.
spy_shares = 110
agg_shares = 200

spy = AlpacaBarset("SPY", "SPDR S&P 500 ETF Trust",              spy_shares, "USD")
agg = AlpacaBarset("AGG", "iShares Core US Aggregate Bond ETF",  agg_shares, "USD")


#### Step 1: In the `Starter_Code` folder, create an environment file (`.env`) to store the values of your Alpaca API key and Alpaca secret key.

#### Step 2: Set the variables for the Alpaca API and secret keys. Using the Alpaca SDK, create the Alpaca `tradeapi.REST` object. In this object, include the parameters for the Alpaca API key, the secret key, and the version number.

In [49]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key    = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2",
)

alpaca


#### Step 3: Set the following parameters for the Alpaca API call:

- `tickers`: Use the tickers for the member’s stock and bond holdings.

- `timeframe`: Use a time frame of one day.

- `start_date` and `end_date`: Use the same date for these parameters, and format them with the date of the previous weekday (or `2020-08-07`). This is because you want the one closing price for the most-recent trading day.


In [50]:
# Set the tickers for both the bond and stock portion of the portfolio
tickers = [spy.ticker, agg.ticker]

# Set timeframe to 1D 
timeframe = "1D"
timezone  = "America/New_York"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2020-08-07", tz=timezone).isoformat()
end_date   = pd.Timestamp("2020-08-07", tz=timezone).isoformat()


#### Step 4: Get the current closing prices for `SPY` and `AGG` by using the Alpaca `get_barset` function. Format the response as a Pandas DataFrame by including the `df` property at the end of the `get_barset` function.

In [51]:
# Use the Alpaca get_barset function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
closing_prices_df = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
).df

# Review the first 5 rows of the Alpaca DataFrame
closing_prices_df.head(5)


AGG                                        SPY  \
                             open    high      low    close   volume    open   
time                                                                           
2020-08-07 00:00:00-04:00  119.66  119.73  119.395  119.445  8830420  333.28   

                                                            
                             high    low   close    volume  
time                                                        
2020-08-07 00:00:00-04:00  334.88  332.3  334.55  98710236

#### Step 5: Navigating the Alpaca response DataFrame, select the `SPY` and `AGG` closing prices, and store them as variables.

In [53]:
gh = closing_prices_df.loc[:,"AGG"].drop(columns="close")

In [61]:
gh.drop(columns="low", inplace=True)

In [66]:
type(["high"])

list

In [20]:
agg.set_response(closing_prices_df)
spy.set_response(closing_prices_df)


In [21]:
# Access the closing price for AGG from the Alpaca DataFrame
# Converting the value to a floating point number
agg_close_price = float(closing_prices_df.loc["2020-08-07",("AGG","close")])

# Print the AGG closing price
agg.closing_price_print()



As of Fri, 2020-Aug-07 (EDT), 
the closing price of the stock and bond portion of the portfolio's
200.0 iShares Core US Aggregate Bond ETF (AGG) is 119.44 USD.
    


In [22]:
# Access the closing price for SPY from the Alpaca DataFrame
# Converting the value to a floating point number
spy_close_price = float(closing_prices_df.loc["2020-08-07",("SPY","close")])

# Print the SPY closing price
spy.closing_price_print()



As of Fri, 2020-Aug-07 (EDT), 
the closing price of the stock and bond portion of the portfolio's
110.0 SPDR S&P 500 ETF Trust (SPY) is 334.55 USD.
    


#### Step 6: Calculate the value, in US dollars, of the current amount of shares in each of the stock and bond portions of the portfolio, and print the results.

In [23]:
# Calculate the current value of the bond portion of the portfolio
agg_value = agg_shares * agg_close_price

# Print the current value of the bond portfolio
agg.closing_value_print()



As of Fri, 2020-Aug-07 (EDT), 
the total value of the stock and bond portion of the portfolio's
200.0 iShares Core US Aggregate Bond ETF (AGG) is 23,889.00 USD.
    


In [24]:
# Calculate the current value of the stock portion of the portfolio
spy_value = spy_shares * spy_close_price

# Print the current value of the stock portfolio
spy.closing_value_print()


As of Fri, 2020-Aug-07 (EDT), 
the total value of the stock and bond portion of the portfolio's
110.0 SPDR S&P 500 ETF Trust (SPY) is 36,800.50 USD.
    


In [25]:
# Calculate the total value of the stock and bond portion of the portfolio
total_stocks_bonds = agg_value + spy_value

# Print the current balance of the stock and bond portion of the portfolio
agg.closing_portfolio_balance_print(agg.collection_time,
                                    total_stocks_bonds,
                                    agg.portfolio,
                                    shares=[agg, spy])



As of Fri, 2020-Aug-07 (EDT),
the total closing value of the stock and bond portion of the portfolio containing 
200.0 iShares Core US Aggregate Bond ETF (AGG) and 110.0 SPDR S&P 500 ETF Trust (SPY) shares 
is 60,689.50 USD.
    


In [26]:
# Calculate the total value of the member's entire savings portfolio
# Add the value of the cryptocurrency walled to the value of the total stocks and bonds
total_portfolio = total_stocks_bonds + total_crypto_wallet

# Print current cryptocurrency wallet balance
agg.closing_portfolio_balance_print(btc.collection_time,
                                    total_portfolio,
                                    "total",
                                    coins=[btc, eth],
                                    shares=[spy, agg])



As of Thu, 2021-Jul-22 23:24:32 (EDT),
the total closing value of the total containing 
110.0 SPDR S&P 500 ETF Trust (SPY) and 200.0 iShares Core US Aggregate Bond ETF (AGG) shares, 
and 1.2 Bitcoin (BTC) and 5.3 Ethereum (ETH) coins 
is 139,030.30 USD.
    


### Evaluate the Emergency Fund

In this section, you’ll use the valuations for the cryptocurrency wallet and for the stock and bond portions of the portfolio to determine if the credit union member has enough savings to build an emergency fund into their financial plan. To do this, complete the following steps:

1. Create a Python list named `savings_data` that has two elements. The first element contains the total value of the cryptocurrency wallet. The second element contains the total value of the stock and bond portions of the portfolio.

2. Use the `savings_data` list to create a Pandas DataFrame named `savings_df`, and then display this DataFrame. The function to create the DataFrame should take the following three parameters:

    - `savings_data`: Use the list that you just created.

    - `columns`: Set this parameter equal to a Python list with a single value called `amount`.

    - `index`: Set this parameter equal to a Python list with the values of `crypto` and `stock/bond`.

3. Use the `savings_df` DataFrame to plot a pie chart that visualizes the composition of the member’s portfolio. The y-axis of the pie chart uses `amount`. Be sure to add a title.

4. Using Python, determine if the current portfolio has enough to create an emergency fund as part of the member’s financial plan. Ideally, an emergency fund should equal to three times the member’s monthly income. To do this, implement the following steps:

    1. Create a variable named `emergency_fund_value`, and set it equal to three times the value of the member’s `monthly_income` of $12000. (You set this earlier in Part 1).

    2. Create a series of three if statements to determine if the member’s total portfolio is large enough to fund the emergency portfolio:

        1. If the total portfolio value is greater than the emergency fund value, display a message congratulating the member for having enough money in this fund.

        2. Else if the total portfolio value is equal to the emergency fund value, display a message congratulating the member on reaching this important financial goal.

        3. Else the total portfolio is less than the emergency fund value, so display a message showing how many dollars away the member is from reaching the goal. (Subtract the total portfolio value from the emergency fund value.)


#### Step 1: Create a Python list named `savings_data` that has two elements. The first element contains the total value of the cryptocurrency wallet. The second element contains the total value of the stock and bond portions of the portfolio.

In [ ]:
# Consolidate financial assets data into a Python list
# YOUR CODE HERE

# Review the Python list savings_data
# YOUR CODE HERE
    

#### Step 2: Use the `savings_data` list to create a Pandas DataFrame named `savings_df`, and then display this DataFrame. The function to create the DataFrame should take the following three parameters:

- `savings_data`: Use the list that you just created.

- `columns`: Set this parameter equal to a Python list with a single value called `amount`.

- `index`: Set this parameter equal to a Python list with the values of `crypto` and `stock/bond`.


In [ ]:
# Create a Pandas DataFrame called savings_df 
# YOUR CODE HERE

# Display the savings_df DataFrame
# YOUR CODE HERE


#### Step 3: Use the `savings_df` DataFrame to plot a pie chart that visualizes the composition of the member’s portfolio. The y-axis of the pie chart uses `amount`. Be sure to add a title.

In [ ]:
# Plot the total value of the member's portfolio (crypto and stock/bond) in a pie chart
# YOUR CODE HERE


#### Step 4: Using Python, determine if the current portfolio has enough to create an emergency fund as part of the member’s financial plan. Ideally, an emergency fund should equal to three times the member’s monthly income. To do this, implement the following steps:

Step 1. Create a variable named `emergency_fund_value`, and set it equal to three times the value of the member’s `monthly_income` of 12000. (You set this earlier in Part 1).

Step 2. Create a series of three if statements to determine if the member’s total portfolio is large enough to fund the emergency portfolio:

* If the total portfolio value is greater than the emergency fund value, display a message congratulating the member for having enough money in this fund.

* Else if the total portfolio value is equal to the emergency fund value, display a message congratulating the member on reaching this important financial goal.

* Else the total portfolio is less than the emergency fund value, so display a message showing how many dollars away the member is from reaching the goal. (Subtract the total portfolio value from the emergency fund value.)


##### Step 4-1: Create a variable named `emergency_fund_value`, and set it equal to three times the value of the member’s `monthly_income` of 12000. (You set this earlier in Part 1).

In [ ]:
# Create a variable named emergency_fund_value
# YOUR CODE HERE


##### Step 4-2: Create a series of three if statements to determine if the member’s total portfolio is large enough to fund the emergency portfolio:

* If the total portfolio value is greater than the emergency fund value, display a message congratulating the member for having enough money in this fund.

* Else if the total portfolio value is equal to the emergency fund value, display a message congratulating the member on reaching this important financial goal.

* Else the total portfolio is less than the emergency fund value, so display a message showing how many dollars away the member is from reaching the goal. (Subtract the total portfolio value from the emergency fund value.)

In [ ]:
# Evaluate the possibility of creating an emergency fund with 3 conditions:
# YOUR CODE HERE
# YOUR CODE HERE
# YOUR CODE HERE


## Part 2: Create a Financial Planner for Retirement

### Create the Monte Carlo Simulation

In this section, you’ll use the MCForecastTools library to create a Monte Carlo simulation for the member’s savings portfolio. To do this, complete the following steps:

1. Make an API call via the Alpaca SDK to get 3 years of historical closing prices for a traditional 60/40 portfolio split: 60% stocks (SPY) and 40% bonds (AGG).

2. Run a Monte Carlo simulation of 500 samples and 30 years for the 60/40 portfolio, and then plot the results.The following image shows the overlay line plot resulting from a simulation with these characteristics. However, because a random number generator is used to run each live Monte Carlo simulation, your image will differ slightly from this exact image:

![A screenshot depicts the resulting plot.](img/5-4-monte-carlo-line-plot.png)

3. Plot the probability distribution of the Monte Carlo simulation. Plot the probability distribution of the Monte Carlo simulation. The following image shows the histogram plot resulting from a simulation with these characteristics. However, because a random number generator is used to run each live Monte Carlo simulation, your image will differ slightly from this exact image:

![A screenshot depicts the histogram plot.](img/5-4-monte-carlo-histogram.png)

4. Generate the summary statistics for the Monte Carlo simulation.



#### Step 1: Make an API call via the Alpaca SDK to get 3 years of historical closing prices for a traditional 60/40 portfolio split: 60% stocks (SPY) and 40% bonds (AGG).

In [ ]:
# Set start and end dates of 3 years back from your current date
# Alternatively, you can use an end date of 2020-08-07 and work 3 years back from that date 
# YOUR CODE HERE
# YOUR CODE HERE

# Set number of rows to 1000 to retrieve the maximum amount of rows
# YOUR CODE HERE

In [ ]:
# Use the Alpaca get_barset function to make the API call to get the 3 years worth of pricing data
# The tickers and timeframe parameters should have been set in Part 1 of this activity 
# The start and end dates should be updated with the information set above
# Remember to add the df property to the end of the call so the response is returned as a DataFrame
# YOUR CODE HERE


# Display both the first and last five rows of the DataFrame
# YOUR CODE HERE
# YOUR CODE HERE


#### Step 2: Run a Monte Carlo simulation of 500 samples and 30 years for the 60/40 portfolio, and then plot the results.

In [ ]:
# Configure the Monte Carlo simulation to forecast 30 years cumulative returns
# The weights should be split 40% to AGG and 60% to SPY.
# Run 500 samples.
# YOUR CODE HERE

# Review the simulation input data
# YOUR CODE HERE


In [ ]:
# Run the Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE


In [ ]:
# Visualize the 30-year Monte Carlo simulation by creating an
# overlay line plot
# YOUR CODE HERE


#### Step 3: Plot the probability distribution of the Monte Carlo simulation.

In [ ]:
# Visualize the probability distribution of the 30-year Monte Carlo simulation 
# by plotting a histogram
# YOUR CODE HERE


#### Step 4: Generate the summary statistics for the Monte Carlo simulation.

In [ ]:
# Generate summary statistics from the 30-year Monte Carlo simulation results
# Save the results as a variable
# YOUR CODE HERE


# Review the 30-year Monte Carlo summary statistics
# YOUR CODE HERE

### Analyze the Retirement Portfolio Forecasts

Using the current value of only the stock and bond portion of the member's portfolio and the summary statistics that you generated from the Monte Carlo simulation, answer the following question in your Jupyter notebook:

-  What are the lower and upper bounds for the expected value of the portfolio with a 95% confidence interval?


In [ ]:
# Print the current balance of the stock and bond portion of the members portfolio
# YOUR CODE HERE


In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_thirty_cumulative_return = # YOUR CODE HERE
ci_upper_thirty_cumulative_return = # YOUR CODE HERE

# Print the result of your calculations
# YOUR CODE HERE


### Forecast Cumulative Returns in 10 Years

The CTO of the credit union is impressed with your work on these planning tools but wonders if 30 years is a long time to wait until retirement. So, your next task is to adjust the retirement portfolio and run a new Monte Carlo simulation to find out if the changes will allow members to retire earlier.

For this new Monte Carlo simulation, do the following: 

- Forecast the cumulative returns for 10 years from now. Because of the shortened investment horizon (30 years to 10 years), the portfolio needs to invest more heavily in the riskier asset&mdash;that is, stock&mdash;to help accumulate wealth for retirement. 

- Adjust the weights of the retirement portfolio so that the composition for the Monte Carlo simulation consists of 20% bonds and 80% stocks. 

- Run the simulation over 500 samples, and use the same data that the API call to Alpaca generated.

- Based on the new Monte Carlo simulation, answer the following questions in your Jupyter notebook:

    - Using the current value of only the stock and bond portion of the member's portfolio and the summary statistics that you generated from the new Monte Carlo simulation, what are the lower and upper bounds for the expected value of the portfolio (with the new weights) with a 95% confidence interval?

    - Will weighting the portfolio more heavily toward stocks allow the credit union members to retire after only 10 years?


In [ ]:
# Configure a Monte Carlo simulation to forecast 10 years cumulative returns
# The weights should be split 20% to AGG and 80% to SPY.
# Run 500 samples.
# YOUR CODE HERE

# Review the simulation input data
# YOUR CODE HERE


In [ ]:
# Run the Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE


In [ ]:
# Visualize the 10-year Monte Carlo simulation by creating an
# overlay line plot
# YOUR CODE HERE


In [ ]:
# Visualize the probability distribution of the 10-year Monte Carlo simulation 
# by plotting a histogram
# YOUR CODE HERE


In [ ]:
# Generate summary statistics from the 10-year Monte Carlo simulation results
# Save the results as a variable
# YOUR CODE HERE


# Review the 10-year Monte Carlo summary statistics
# YOUR CODE HERE

### Answer the following questions:

#### Question: Using the current value of only the stock and bond portion of the member's portfolio and the summary statistics that you generated from the new Monte Carlo simulation, what are the lower and upper bounds for the expected value of the portfolio (with the new weights) with a 95% confidence interval?

In [ ]:
# Print the current balance of the stock and bond portion of the members portfolio
# YOUR CODE HERE


In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_ten_cumulative_return = # YOUR CODE HERE
ci_upper_ten_cumulative_return = # YOUR CODE HERE

# Print the result of your calculations
# YOUR CODE HERE


#### Question: Will weighting the portfolio more heavily to stocks allow the credit union members to retire after only 10 years?